In [39]:
import pandas as pd
import numpy as np
from scipy.spatial.distance import euclidean
from fastdtw import fastdtw
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from tsfresh import select_features
from tsfresh.utilities.dataframe_functions import impute
from tsfresh import extract_features

df = pd.read_csv('signs_all_ts.csv')
df2 = pd.read_csv('signs_flat.csv')

In [40]:
df = df.drop(['Unnamed: 0'], axis=1)

In [41]:
df.ID = df.ID.astype(str)
df['ID'] = df['folder'] + df['label'] + df['ID']

In [42]:
g = df.groupby('ID')
df['ix'] = df.index.values

In [43]:
df['frame'] = g['ix'].rank(method='first')

In [44]:
df['frame'] = df['frame'].astype(int)

In [45]:
y = df2['label']
df.drop(['label', 'folder', 'ix'], axis = 1, inplace=True)

In [ ]:
extracted_features = extract_features(df, column_id="ID", column_sort="frame")

Feature Extraction:   0%|                                                                       | 0/22 [00:00<?, ?it/s]

In [ ]:
impute(extracted_features)
features_filtered = select_features(extracted_features, y)